In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/achi-mapping/ACHI_WITH_BLOCK_MBS.xlsx
/kaggle/input/achi-mapping/Y Hospital.xls


In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import nltk
import sklearn
import string
from nltk import pos_tag 
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.isri import ISRIStemmer
from tensorflow.python.keras.preprocessing.text import Tokenizer 
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM ,GRU
from keras.layers.embeddings import Embedding


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [3]:
data = pd.read_excel('/kaggle/input/achi-mapping/Y Hospital.xls',header=1)

In [4]:
data.head()

,Service Code,Service Name,Service Type,Price
0,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,47.25
1,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,7575.00
2,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal,12755.00
3,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal,8558.00
4,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,13550.00


In [5]:
service_name = pd.DataFrame(data['Service Name'])

In [6]:
achi = pd.read_excel('/kaggle/input/achi-mapping/ACHI_WITH_BLOCK_MBS.xlsx')

In [7]:
achi.head()

,Serv,ACHI,MBS,Block
0,Achillotenotomy 49718-01 [1542],Achillotenotomy,49718-01,[1542]
1,Acromioplasty (open) 48903-00 [1400],Acromioplasty (open),48903-00,[1400]
2,Acupuncture 93173-00 [1908],Acupuncture,93173-00,[1908]
3,"Adaptometry, dark 11211-00 [1831]","Adaptometry, dark",11211-00,[1831]
4,Adenoidectomy 41801-00 [412],Adenoidectomy,41801-00,[412]


In [8]:
imp = ['against','not','don',"don't","aren't","couldn't","didn't","doesn't","hadn't","hasn't","haven't","isn't","mightn't","needn't","wasn't","weren't","wouldn't"]
stop_words = [w for w in stopwords.words('english') if w not in imp]
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    text = [word.translate(str.maketrans('', '', string.punctuation)) for word in text]
    # remove words that contain numbers
    #text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stop_words
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    #pos_tags = pos_tag(text)
    # lemmatize text
    #text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    text = [re.sub('\d','',t) for t in text]
    
    # join all
    text = " ".join(text)
    text = text.replace('day pkg','')
    text = text.replace('days','')
    return(text)

In [9]:
cleaned_service_name = service_name['Service Name'].apply(lambda x :clean_text(x))

In [10]:
cleaned_ACHI = achi.ACHI.apply(lambda x : clean_text(x))

In [11]:
cleaned_ACHI

0                                          achillotenotomy
1                                       acromioplasty open
2                                              acupuncture
3                                         adaptometry dark
4                                            adenoidectomy
                               ...                        
20754    loose body joint knee open   arthroscopic clos...
20755    loose body joint knee open   arthroscopic clos...
20756    repair joint also arthroplasty temporomandibul...
20757    repair joint also arthroplasty temporomandibul...
20758    tomography nec   computerised axial ct quantit...
Name: ACHI, Length: 20759, dtype: object

In [12]:
cleaned_service_name

0                                     lanoxinmg injections
1        endoscopic retrograde cholangiopacreatography ...
2                                         cholecystectomy 
3                       unilateral repair inguinal hernia 
4          unilateral inguinal herniorrhaphy laparoscopic 
                               ...                        
12478                                    lipidol injection
12479                                      lipofundine  ml
12480                                 marcaine  heavy ml s
12481                                 methylene blue ml ml
12482                         avalox  mg infusion solution
Name: Service Name, Length: 12483, dtype: object

In [13]:
corpus = pd.concat([cleaned_service_name,cleaned_ACHI])

In [14]:
print(corpus)

0                                     lanoxinmg injections
1        endoscopic retrograde cholangiopacreatography ...
2                                         cholecystectomy 
3                       unilateral repair inguinal hernia 
4          unilateral inguinal herniorrhaphy laparoscopic 
                               ...                        
20754    loose body joint knee open   arthroscopic clos...
20755    loose body joint knee open   arthroscopic clos...
20756    repair joint also arthroplasty temporomandibul...
20757    repair joint also arthroplasty temporomandibul...
20758    tomography nec   computerised axial ct quantit...
Length: 33242, dtype: object


In [15]:
vect = TfidfVectorizer().fit(corpus)

In [16]:
achi_vect = vect.transform(cleaned_ACHI)
sn_vect = vect.transform(cleaned_service_name)
sn_vect.shape,achi_vect.shape

((12483, 13648), (20759, 13648))

In [17]:
# def normalize(word_vec):
#     norm=np.linalg.norm(word_vec)
#     if norm == 0:
#         return word_vec
#     return word_vec/norm

In [18]:
#for i in sn_vect.A:
 #   i = normalize(i)


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
def find_similar(tfidf_data,tfidf_ref, index, rank = 1):
    #multiplying the 2 matrix to find the cosine similarity
    cosine_similarities = linear_kernel(tfidf_data[index:index+1], tfidf_ref[:]).flatten()
    #getting the most similar index
    highest_ind = cosine_similarities.argsort()[::-1][rank]
    #return the index along with its similarity
    return highest_ind, cosine_similarities[highest_ind]

In [20]:
from sklearn.metrics.pairwise import linear_kernel
result = pd.DataFrame(columns = ['Service Code','Service Name','Service Type','ACHI','Similarity','MBS','BLOCK'])
resultAll = pd.DataFrame(columns = ['Service Code','Service Name','Service Type','ACHI','Similarity','MBS','BLOCK'])
for i in range(len(service_name)):
    index, simlarity = find_similar(sn_vect, achi_vect, i, 1)
    result.loc[i] = [data.iloc[i][0],data.iloc[i][1],data.iloc[i][2], achi.iloc[index][1], simlarity,achi.iloc[index][2],achi.iloc[index][3]]
    resultAll = resultAll.append(result)
    result.drop(result.index, inplace = True)

In [21]:
resultAll.head()

,Service Code,Service Name,Service Type,ACHI,Similarity,MBS,BLOCK
0,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,"Arthroplasty ( Repair, joint) 50127-00 [1571]...",0.000000,45871-00,[1362]
1,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,Cholangiopancreatography ___endoscopic retrogr...,0.555171,30484-00,[957]
2,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal,Cholecystectomy 30443-00 [965] ___laparoscopic,0.831226,30445-00,[965]
3,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,0.735025,30609-02,[990]
4,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,Repair ___hernia ___inguinal ___unilateral (op...,0.337849,30609-02,[990]


In [22]:
len(resultAll[resultAll.Similarity > 0.5])/len(resultAll)

0.08435472242249459

In [23]:
resultAll.ACHI = resultAll.ACHI.apply(lambda x:x.replace('_',''))

In [24]:
resultAll.ACHI = resultAll.ACHI.apply(lambda x : re.sub('\d','',x))

In [25]:
resultAll.ACHI = resultAll.ACHI.apply(lambda x :x.replace('[]',''))

In [26]:
resultAll.head(50)

,Service Code,Service Name,Service Type,ACHI,Similarity,MBS,BLOCK
0,550210232,LANOXIN-0.5MG INJECTION-1'S,Pharmacy,"Arthroplasty ( Repair, joint) - temporomandi...",0.000000,45871-00,[1362]
1,07300199,ENDOSCOPIC RETROGRADE CHOLANGIO-PACREATOGRAPHY...,Package Deal,Cholangiopancreatography endoscopic retrograde...,0.555171,30484-00,[957]
2,07300200,CHOLECYSTECTOMY - 5 DAY PKG,Package Deal,Cholecystectomy - laparoscopic,0.831226,30445-00,[965]
3,07300201,UNILATERAL REPAIR OF INGUINAL HERNIA - 1 DAY PKG,Package Deal,Repair hernia inguinal unilateral (open) (with...,0.735025,30609-02,[990]
4,07300202,UNILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPI...,Package Deal,Repair hernia inguinal unilateral (open) (with...,0.337849,30609-02,[990]
5,07300203,UNILATERAL REPAIR OF INGUINAL HERNIA W/ GRAFT ...,Package Deal,Repair hernia inguinal unilateral (open) (with...,0.848405,30609-02,[990]
6,07300205,BILATERAL INGUINAL HERNIORRHAPHY (LAPAROSCOPIC...,Package Deal,Repair hernia inguinal bilateral (open) (with ...,0.325405,30609-03,[990]
7,07300206,BILATERAL REPAIR OF INGUINAL HERNIA W/ GRAFT O...,Package Deal,Repair hernia inguinal bilateral (open) (with ...,0.849781,30609-03,[990]
8,07300207,UNILATERAL REPAIR OF FEMORAL HERNIA- 1 DAY PKG,Package Deal,Repair hernia femoral unilateral (open) (with ...,0.714569,30609-00,[991]
9,07300208,BILATERAL REPAIR OF FEMORAL HERNIA - 2 DAY PKG,Package Deal,Repair hernia femoral bilateral (open) (with g...,0.717600,30609-01,[991]


In [27]:
resultAll.sort_values(by=['Similarity'],ascending=False)

,Service Code,Service Name,Service Type,ACHI,Similarity,MBS,BLOCK
898,0730001081,TEMPORAL ARTERY BIOPSY -1 DAY PKG,Package Deal,Biopsy temporal artery,1.000000,34109-00,[699]
74,07300139,INCISION OF PILONIDAL SINUS OR CYST - 1 DAY PKG,Package Deal,Incision sinus pilonidal (cyst),1.000000,30676-00,[1659]
10198,07300078,SEPTOPLASTY W/ SUBMUCOUS RESECTION OF NASAL SE...,Package Deal,Resection also Excision nasal septum (submuc...,0.961032,41671-03,[379]
1658,07300865,FOREHEAD LIFT -1DAY PKG.,Package Deal,Lift forehead (bilateral),0.922847,45588-01,[1675]
7240,450101905,LEFT VENTRICULAR ASSIST DEVICE,Other Medical Services,Removal also Excision device ventricular ass...,0.914414,38621-00,[608]
19,07300222,UNILATERAL SIMPLE MASTECTOMY 3 DAY PKG,Package Deal,Mastectomy simple (unilateral) - bilateral,0.911044,31518-01,[1748]
3463,05100194,EPIDURAL BLOOD PATCH,Surgical Procedure,Patch spinal (epidural) blood,0.899046,18233-00,[38]
8605,07300548,UNILATERAL ORCHIDECTOMY - 1 DAY PKG,Package Deal,Orchidectomy (unilateral) - bilateral,0.894238,30641-01,[1184]
3615,050050136,NASAL PACKING,Other Medical Services,Replacement packing nasal,0.890586,92030-00,[1887]
824,07300450,PYLOROPLASTY - 5 DAY PKG,Package Deal,Reconstruction pyloroplasty,0.885168,30517-00,[888]


In [ ]:
resultAll.to_csv('Y_hospital.csv',index=0)